# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt
# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Dsc
0,0,Escanaba,US,45.7453,-87.0646,32.31,96,100,5.75,mist
1,1,Taby,SE,59.4439,18.0687,42.58,93,100,8.05,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,46.06,87,0,4.61,clear sky
3,3,Jumla,NP,29.2747,82.1838,51.89,47,93,2.10,overcast clouds
4,4,Hambantota,LK,6.1241,81.1185,77.34,100,75,1.14,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temprature criteria?"))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is maximum temprature criteria?"))

What is the minimum temprature criteria?50
What is maximum temprature criteria?90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Dsc
3,3,Jumla,NP,29.2747,82.1838,51.89,47,93,2.10,overcast clouds
4,4,Hambantota,LK,6.1241,81.1185,77.34,100,75,1.14,broken clouds
5,5,Bathsheba,BB,13.2167,-59.5167,79.41,89,75,27.63,broken clouds
6,6,Georgetown,MY,5.4112,100.3354,80.53,93,20,3.44,few clouds
8,8,Nouakchott,MR,18.0858,-15.9785,73.33,60,0,12.77,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Dsc
3,3,Jumla,NP,29.2747,82.1838,51.89,47,93,2.10,overcast clouds
4,4,Hambantota,LK,6.1241,81.1185,77.34,100,75,1.14,broken clouds
5,5,Bathsheba,BB,13.2167,-59.5167,79.41,89,75,27.63,broken clouds
6,6,Georgetown,MY,5.4112,100.3354,80.53,93,20,3.44,few clouds
8,8,Nouakchott,MR,18.0858,-15.9785,73.33,60,0,12.77,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City","Country","Max Temp","Weather Dsc","Lat","Lng"]].copy()

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather Dsc,Lat,Lng
3,Jumla,NP,51.89,overcast clouds,29.2747,82.1838
4,Hambantota,LK,77.34,broken clouds,6.1241,81.1185
5,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167
6,Georgetown,MY,80.53,few clouds,5.4112,100.3354
8,Nouakchott,MR,73.33,clear sky,18.0858,-15.9785


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather Dsc,Lat,Lng,Hotel Name
3,Jumla,NP,51.89,overcast clouds,29.2747,82.1838,
4,Hambantota,LK,77.34,broken clouds,6.1241,81.1185,
5,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167,
6,Georgetown,MY,80.53,few clouds,5.4112,100.3354,
8,Nouakchott,MR,73.33,clear sky,18.0858,-15.9785,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Weather Dsc,Lat,Lng,Hotel Name
3,Jumla,NP,51.89,overcast clouds,29.2747,82.1838,Royal Garden Hotel
4,Hambantota,LK,77.34,broken clouds,6.1241,81.1185,Bungalow 63
5,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167,Atlantis Hotel
6,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
8,Nouakchott,MR,73.33,clear sky,18.0858,-15.9785,HOTEL IMAN


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Weather Dsc,Lat,Lng,Hotel Name
3,Jumla,NP,51.89,overcast clouds,29.2747,82.1838,Royal Garden Hotel
4,Hambantota,LK,77.34,broken clouds,6.1241,81.1185,Bungalow 63
5,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167,Atlantis Hotel
6,Georgetown,MY,80.53,few clouds,5.4112,100.3354,Cititel Penang
8,Nouakchott,MR,73.33,clear sky,18.0858,-15.9785,HOTEL IMAN
...,...,...,...,...,...,...,...
697,Deori,IN,68.86,clear sky,21.4500,82.6167,बंजारी डीपा
698,Banyo,CM,66.47,scattered clouds,6.7500,11.8167,Motel JOMBA
701,Sao Francisco Do Sul,BR,70.30,scattered clouds,-26.2433,-48.6381,Hotel Kontiki
703,Mizdah,LY,52.29,clear sky,31.4451,12.9801,محمدالمبروك الاريقط


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
#clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [14]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Dsc} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [15]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Royal Garden Hotel</dd>\n<dt>City</dt><dd>Jumla</dd>\n<dt>Country</dt><dd>NP</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 51.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bungalow 63</dd>\n<dt>City</dt><dd>Hambantota</dd>\n<dt>Country</dt><dd>LK</dd>\n<dt>Current Weather</dt><dd>broken clouds and 77.34 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Atlantis Hotel</dd>\n<dt>City</dt><dd>Bathsheba</dd>\n<dt>Country</dt><dd>BB</dd>\n<dt>Current Weather</dt><dd>broken clouds and 79.41 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Cititel Penang</dd>\n<dt>City</dt><dd>Georgetown</dd>\n<dt>Country</dt><dd>MY</dd>\n<dt>Current Weather</dt><dd>few clouds and 80.53 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>HOTEL IMAN</dd>\n<dt>City</dt><dd>Nouakchott</dd>\n<dt>Country</dt><dd>MR</dd>\n<dt>Current Weather</dt><dd>clear sky and 73.33 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Camacupa</dd>\n<d

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [16]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
3,29.2747,82.1838
4,6.1241,81.1185
5,13.2167,-59.5167
6,5.4112,100.3354
8,18.0858,-15.9785
9,-12.0167,17.4833
11,14.8961,-24.4956
12,-33.0117,17.9442
13,61.9875,5.1352
15,28.6585,-17.9182


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [17]:
# Add a marker layer for each city to the map. 
# YOUR CODE HERE
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
# YOUR CODE HERE
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure containing the map
# YOUR CODE HERE
fig

Figure(layout=FigureLayout(height='420px'))